In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate, Input, Flatten
from keras.models import Model
from keras import optimizers
from keras import regularizers
import json
import re

Using TensorFlow backend.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pro

In [2]:
def jsons_name_type(jsons_name):
    if not jsons_name:
        return -1
    if "Цифра дня" in jsons_name:
        return 1
    if "AliExpress" in jsons_name:
        return 2 
    if "Тинькофф" in jsons_name or "Tinkoff" in jsons_name:
        return 3
    if "ЧМ" in jsons_name:
        return 4
    if "фильм"  in jsons_name or "смотрет"  in jsons_name:
        return 5
    if "кредит"  in jsons_name:
        return 6
    if "инвестиц"  in jsons_name:
        return 7
    if "₽"  in jsons_name or "$"  in jsons_name:
        return 8
    if "тпуск"  in jsons_name:
        return 9
    return 0

In [3]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
stories_description = pd.read_csv(f'{data_path}stories_description.csv')
stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [4]:
stories_reaction_train['reaction_hour']=stories_reaction_train.event_dttm.dt.hour
stories_reaction_test['reaction_hour']=stories_reaction_test.event_dttm.dt.hour
stories_reaction_train['reaction_dayofweek']=stories_reaction_train.event_dttm.dt.dayofweek
stories_reaction_test['reaction_dayofweek']=stories_reaction_test.event_dttm.dt.dayofweek

stories_description['jsons'] = stories_description.story_json.map(lambda x:json.loads(x.replace("\\\\", "\\")))
stories_description['jsons_name'] =stories_description.jsons.map(lambda x:x['properties'].get('name'))
stories_description['texts'] =stories_description.jsons.map(lambda x:re.findall(""""text": \"([^\"]*)\",""", json.dumps(x, ensure_ascii=False)))
stories_description['jsons_name_type'] = stories_description.jsons_name.map(lambda x:jsons_name_type(x))

In [5]:
train = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
train = train.merge(stories_description, on='story_id', how='left')

val = stories_reaction_test.merge(customer_test, on='customer_id', how='left')
val = val.merge(stories_description, on='story_id', how='left')

train=train.fillna(0)
val=val.fillna(0)
# train_and_val.fillnull(0)


In [6]:
train_and_val = pd.concat([train,val])

/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
len(train), len(val) , len(train_and_val) 

(473141, 172049, 645190)

In [8]:

train['gender_cd']=train.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
val['gender_cd']=val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
train['age']=train.age/100
val['age']=val.age/100

In [10]:
def to_OHE(field):
    count = len(set(train_and_val[field].values))
    d = dict(zip(list(set(train_and_val[field].values)), range(1, count+1)))
    train[field]=train[field].map(lambda x:  d[x])
    val[field]=val[field].map(lambda x:  d[x])
    return count
customers_count = to_OHE('customer_id')
marital_status_count = to_OHE('marital_status_cd')
stories_count = to_OHE('story_id')
job_position_count = to_OHE('job_position_cd')
job_title_count = to_OHE('job_title')
jsons_name_type_count = to_OHE('jsons_name_type')




In [11]:
values = set(train_and_val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values.reshape(-1))
producs_count = len(values)
d = dict(zip(list(values), range(1, len(values)+1)))
for f in ['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']:
    train[f]=train[f].map(lambda x:  d[x])
    val[f]=val[f].map(lambda x:  d[x])

In [12]:
# customers_count = len(set(train_and_val.customer_id.values))
# marital_status_count = len(set(train_and_val.marital_status_cd.values))
# stories_count = len(set(train_and_val.story_id.values))
# job_position_count = len(set(train_and_val.job_position_cd.values))
# job_position_count = len(set(train_and_val.job_position_cd.values))


# customers_dict = dict(zip(sorted(list(set(train_and_val.customer_id.values))), range(1, customers_count+1)))
# stories_dict = dict(zip(sorted(list(set(train_and_val.story_id.values))), range(1, stories_count+1)))
# job_position_dict = dict(zip(sorted(list(set(train_and_val.job_position_cd.values))), range(1, job_position_count+1)))
# marital_status_dict = dict(zip(sorted(list(set(train_and_val.marital_status_cd.values))), range(1, marital_status_count+1)))

# train['customer_id']=train.customer_id.map(lambda x:  customers_dict[x])
# train['story_id']=train.story_id.map(lambda x:  stories_dict[x])
# train['job_position_cd']=train.job_position_cd.map(lambda x:  job_position_dict[x])
# train['marital_status_cd']=train.marital_status_cd.map(lambda x:  marital_status_dict[x])

# val['customer_id']=val.customer_id.map(lambda x:  customers_dict[x])
# val['story_id']=val.story_id.map(lambda x:  stories_dict[x])
# val['job_position_cd']=val.job_position_cd.map(lambda x:  job_position_dict[x])
# val['marital_status_cd']=val.marital_status_cd.map(lambda x:  marital_status_dict[x])


In [13]:
train.first_session_dttm[50000:]

50000     2013-12-30 18:03:33
50001     2013-12-30 18:03:33
50002     2013-12-30 18:03:33
50003     2013-12-30 18:03:33
50004     2013-12-30 18:03:33
                 ...         
473136                      0
473137                      0
473138                      0
473139                      0
473140                      0
Name: first_session_dttm, Length: 423141, dtype: object

In [14]:
import keras.backend as K
def metric_loss(batch_size):
    def the_loss(y_true, y_pred):

        return -(len(val.customer_id.values)/batch_size)*K.sum(y_true*y_pred)/40441.4
    return(the_loss)
# /K.mean(K.abs(y_true))
# /K.sum(K.abs(y_true))

#     return -K.sum(y_true*y_pred)/K.sum(K.abs(y_true))
#     return K.sum(y_true*[10,0.5,0.1,0.1]*y_pred)/K.sum(y_true*[-10,0.5,0.1,-0.1])

In [15]:
def reaction_to_class(reaction):
    if reaction=="dislike":
        return -10
    if reaction=="like":
        return 0.5
    if reaction=="view":
        return 0.1
    if reaction=="skip":
        return -0.1
trainY = np.array(list(train.event.map(lambda x: reaction_to_class(x)).values))
valY = np.array(list(val.event.map(lambda x: reaction_to_class(x)).values))

AttributeError: 'DataFrame' object has no attribute 'event'

In [16]:
model_input_customer = Input((1,), name='customer_id')
x1 = Embedding(customers_count, 4, input_length=1)(model_input_customer)
x1 = Flatten()(x1)

model_input_story = Input((1,), name='story_id')
x2 = Embedding(stories_count, 2, input_length=1)(model_input_story)
x2 = Flatten()(x2)

# model_input_job = Input((1,), name='job_id')
# x3 = Embedding(job_position_count, 1, input_length=1)(model_input_job)
# x3 = Flatten()(x3)


model_input_job_title = Input((1,), name='job_title_id')
x4 = Embedding(job_title_count, 2, input_length=1)(model_input_job_title)
x4 = Flatten()(x4)

# model_input_products = Input((7,), name='producs')
# x5 = Embedding(producs_count, 8, input_length=7)(model_input_products)
# x5 = Flatten()(x5)

model_input_story_name = Input((1,), name='story_name_id')
x6 = Embedding(jsons_name_type_count, 2, input_length=1)(model_input_story_name)
x6 = Flatten()(x6)

model_input_3 = Input((2,), name='customer_data')
# model_input_4 = Input((2,), name='customer_data2')

x = Concatenate()([x1, x2, x4, x6, model_input_3])

x = Dense(8, activation='tanh', 
#           kernel_regularizer=regularizers.l2(0.0000001)
         )(x)
# x = Dense(64, activation='tanh')(x)

# x = Dropout(0.5)(x)


# x = Concatenate()([x, model_input_3])

# x = Dense(4, activation='tanh')(x)


x = Dense(1, activation='tanh')(x)
model = Model([model_input_customer, 
               model_input_story, 
               model_input_story_name,
               model_input_job_title,
#                model_input_job, 
               model_input_3,
#                model_input_4, 
#                model_input_products
              ], x)
# model = Model(model_input, x)





print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
customer_id (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
story_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
job_title_id (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
story_name_id (InputLayer)      (None, 1)            0                                            
____________________________________________________________________________________________

In [17]:
len(val.customer_id.values)

172049

In [18]:
# batch_size=320248
batch_size=64
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss=metric_loss(batch_size))


In [19]:


model.fit(
    shuffle=True,
#     x = [train.customer_id.values, train.story_id.values, train[['age', 'gender_cd']].values],
        x = [train.customer_id.values,
             train.story_id.values,
             train.jsons_name_type.values, 
             train.job_title.values,
#              train.job_position_cd.values,
             train[['age', 'gender_cd']].values,
#              train[['reaction_hour', 'reaction_dayofweek']].values/30,
#              train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
            ], 
# steps_per_epoch=1000,
          y = trainY,
          batch_size=batch_size,
          epochs=30,
#     validation_data = ([val.customer_id.values, val.story_id.values, val[['age', 'gender_cd']].values], valY),
#           validation_data=([val.customer_id.values, 
#                             val.story_id.values, 
#                             val.jsons_name_type.values, 
#                             val.job_title.values,
# #                             val.job_position_cd.values,
#                             val[['age', 'gender_cd']].values,
# #                             val[['reaction_hour', 'reaction_dayofweek']].values/30,
# #                             val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
#                            ], valY),
         )

# model.fit(x = train.customer_id.values, 
#           y = trainY,
# #           steps_per_epoch=1000,
#           batch_size=batch_size,
#           epochs=100,
#           validation_data=(val.customer_id.values, valY),
# #           validation_steps=len(val.customer_id.values)/batch_size
#          )
# 320248/320248 [==============================] - 0s 0us/step - loss: -0.2332 - val_loss: -0.2353


Epoch 1/30
473141/473141 [==============================] - 16s 33us/step - loss: -0.4166
Epoch 2/30
473141/473141 [==============================] - 15s 32us/step - loss: -0.6406
Epoch 3/30
473141/473141 [==============================] - 15s 32us/step - loss: -0.6835
Epoch 4/30
 29120/473141 [>.............................] - ETA: 14s - loss: -0.7631

KeyboardInterrupt: 

In [20]:

val_predict = model.predict([val.customer_id.values,
                             val.story_id.values,
                             val.jsons_name_type.values,
                             val.job_title.values,
#                              val.job_position_cd.values,
                             val[['age', 'gender_cd']].values,
#                              val[['reaction_hour', 'reaction_dayofweek']].values/30,
#                              val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
                            ])
# train_predict = model.predict(train.customer_id.values)
# val_predict = model.predict(val.customer_id.values)

In [22]:
val['score'] = val_predict

In [25]:
val[['answer_id','score']].to_csv('predict2.csv', index=False)